In [1]:
from featrixclient import Featrix
import uuid

fc = Featrix(url="https://stage.featrix.com", client_id="65f9fc5c94e2d92af20ff47b", client_secret="mtr_t4of2fHmlOs8tD0ABJi/0CNPCTeqYzSdx+v2zX/nEEQ=")

projects = fc.projects()


/Users/mtr/.python/fc_os/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
for p in projects:
    print(f"Project: {p.name} ({p.id})")

Project: Testing FC 383f3a65-d7bb-4ffc-8da5-5afce0181887 (66b3ba666b2d29b4e5b80b5a)
Project: Testing FC 67488b81-2c94-4176-b3e8-3e6a52eed356 (66b3b75cfcba35085277c8fc)
Project: Testing FC (66b3b30dfcba35085277c8f9)
Project: jul11-two (668fe64a9ed407f5cb6fe419)
Project: jul9final (668dcee4d69f6f60bee3e141)
Project: ddd (668d7a1c9cdacf6386bb3581)
Project: flex (668009b321b9c3d2121087b4)


In [2]:
project = fc.create_project(f"Testing FC {uuid.uuid4()}")
upload = fc.get_upload(filename="animal-height-weight-5000.csv")
project = project.associate(upload)
es = fc.create_embedding_space(project, "Testing Embedding Space")
jobs = es.get_jobs()
# Should be 1 here -- the training job (there is also a create projection, but that isnt' a training job...
assert(1 == len(jobs))
assert(2 == len(es.get_jobs(active=True, training=False)))
training = jobs[0]
print(f"Job {training.job_type} ({training.id}) started")
training.wait_for_completion("Training...")


Training... Running: training;    100.0% complete (00:00:00 remaining); epoch = 5/5; current loss = 0.268


FeatrixConnectionError: Connection error for URL https://stage.featrix.com/api/neural/job/66b3b770fd34ec26b6c74644: __request error: ('Connection aborted.', TimeoutError(60, 'Operation timed out'))__

In [3]:
from featrixclient.featrix_embedding_space import FeatrixEmbeddingSpace

project = fc.create_project(f"Testing FC {uuid.uuid4()}")
# Use upload = fc.upload_file("path/to/filename") if its not already uploaded. 
upload = fc.get_upload(filename="animal-height-weight-5000.csv")
project = project.associate(upload)
nf = fc.create_neural_function(target_fields="Height", project=project)
es = FeatrixEmbeddingSpace.by_id(nf.embedding_space_id, fc)
es_jobs = es.get_jobs()
nf_jobs = nf.get_jobs()
assert(1 == len(es_jobs))
assert(1 == len(nf_jobs))
# wait for the es
es_jobs[0].wait_for_completion("Embedding Space Training...")
nf_jobs[0].wait_for_completion("Neural Function Training...")




Called project.ready(False
Called project.ready(False
Initially the not ready count is 0
ES jobs 1:


In [3]:
project = projects[0]
embeddings = project.embedding_spaces()
print(f"project {project.id} embeddings: {embeddings[0].project_id} {len(embeddings)}")
nfs = project.neural_functions(embedding_space=embeddings[0])

project 66b3ba666b2d29b4e5b80b5a embeddings: 66b3ba666b2d29b4e5b80b5a 1


In [ ]:
es_jobs = embeddings[0].get_jobs()
nf_jobs = nfs[0].get_jobs()
es_jobs[0].wait_for_completion("Embedding...")
nf_jobs[0].wait_for_completion("Neural Function...")

Embedding... Running: training;     98.0% complete (00:00:27 remaining); epoch = 75/75; current loss = 0.086


In [7]:
results = fc.api.op("es_get_models", embedding_space_id=str(es.id))

FeatrixConnectionError: Connection error for URL https://stage.featrix.com/api/neural/embedding_space/66b3ba686b2d29b4e5b80b5d/models/: __No more retries, multiple errors __

In [20]:
jobs = project.jobs()
print(jobs[0].model_dump_json(indent=2))

{
  "id": "66b3b4916b2d29b4e5b80b59",
  "created_by": "65f9fc5c94e2d92af20ff47b",
  "created_at": "2024-08-07T17:53:21.010000",
  "updated_at": "2024-08-07T17:53:21.203000",
  "organization_id": "65f9fc5a94e2d92af20ff47a",
  "job_type": "embedding-space-projection-create",
  "readonly_object": false,
  "project_id": "66b3b30dfcba35085277c8f9",
  "embedding_space_id": "66b3b4906b2d29b4e5b80b57",
  "model_id": null,
  "upload_ids": null,
  "finished": false,
  "error": false,
  "message": null,
  "error_msg": null,
  "error_details": null,
  "error_time": null,
  "exception": null,
  "warnings": [],
  "start_time": null,
  "end_time": null,
  "celery_id": "355a2cbb-9e74-4a9b-98f9-8bc01688010d",
  "changed_ids": null,
  "finished_stats": null,
  "request_args": {
    "job_type": "embedding-space-projection-create",
    "skip_auto_load": false,
    "project_id": "66b3b30dfcba35085277c8f9",
    "embedding_space_id": "66b3b4906b2d29b4e5b80b57"
  },
  "incremental_status": null,
  "last_updat

In [22]:
p = es.fc.get_project_by_id(es.project_id)

jobs = p.jobs()
es_jobs = []
for job in jobs:
    if str(job.embedding_space_id) == str(es.id):
        es_jobs.append(job)
print(es_jobs)


[FeatrixJob(id='66b3b4916b2d29b4e5b80b59', created_by=ObjectId('65f9fc5c94e2d92af20ff47b'), created_at=datetime.datetime(2024, 8, 7, 17, 53, 21, 10000), updated_at=datetime.datetime(2024, 8, 7, 17, 53, 21, 203000), organization_id=ObjectId('65f9fc5a94e2d92af20ff47a'), job_type=<JobType.JOB_TYPE_ES_CREATE_PROJECTION: 'embedding-space-projection-create'>, readonly_object=False, project_id=ObjectId('66b3b30dfcba35085277c8f9'), embedding_space_id=ObjectId('66b3b4906b2d29b4e5b80b57'), model_id=None, upload_ids=None, finished=False, error=False, message=None, error_msg=None, error_details=None, error_time=None, exception=None, warnings=[], start_time=None, end_time=None, celery_id='355a2cbb-9e74-4a9b-98f9-8bc01688010d', changed_ids=None, finished_stats=None, request_args={'job_type': 'embedding-space-projection-create', 'skip_auto_load': False, 'project_id': '66b3b30dfcba35085277c8f9', 'embedding_space_id': '66b3b4906b2d29b4e5b80b57'}, incremental_status=None, last_updated=datetime.datetime(

In [25]:
from featrixclient.models.job_type import JobType
nj = []
active = training = True
for job in es_jobs:
    if active or training:
        if active and job.finished:
            continue
        if training and job.job_type not in [JobType.JOB_TYPE_ES_CREATE, JobType.JOB_TYPE_ES_TRAIN_MORE]:
            continue
    nj.append(job)
print(nj)
        

[FeatrixJob(id='66b3b4906b2d29b4e5b80b58', created_by=ObjectId('65f9fc5c94e2d92af20ff47b'), created_at=datetime.datetime(2024, 8, 7, 17, 53, 20, 999000), updated_at=datetime.datetime(2024, 8, 7, 17, 58, 14, 806000), organization_id=ObjectId('65f9fc5a94e2d92af20ff47a'), job_type=<JobType.JOB_TYPE_ES_CREATE: 'embedding-space-create'>, readonly_object=False, project_id=ObjectId('66b3b30dfcba35085277c8f9'), embedding_space_id=ObjectId('66b3b4906b2d29b4e5b80b57'), model_id=None, upload_ids=None, finished=False, error=False, message=None, error_msg=None, error_details=None, error_time=None, exception=None, warnings=[], start_time=datetime.datetime(2024, 8, 7, 17, 53, 28, 416000), end_time=None, celery_id=None, changed_ids=None, finished_stats=None, request_args={'job_type': 'embedding-space-create', 'skip_auto_load': False, 'transient_upload_file_name': None, 'upload_ids': None, 'project_id': '66b3b30dfcba35085277c8f9', 'embedding_space_id': '66b3b4906b2d29b4e5b80b57', 'name': 'Testing Embed